<a href="https://colab.research.google.com/github/lazybuttrying/TensorFlow-Certificate/blob/main/C3/W2/ungraded_labs/C3_W2_Lab_1_imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

**Note:** This notebook can run using TensorFlow 2.5.0

In [ ]:
#!pip install tensorflow==2.5.0

In [ ]:
import tensorflow as tf
print(tf.__version__)

# !pip install -q tensorflow-datasets

2.6.0


In [ ]:
# tensorflow에서 제공하는 데이터 중 imdb 리뷰 데이터를 이용

import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

In [ ]:
import numpy as np

# split data
train_data, test_data = imdb['train'], imdb['test']

training_sentences = [] 
training_labels = [] # 1 = positive , 0 = negative

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s,l in train_data:
  training_sentences.append(s.numpy().decode('utf8'))
  training_labels.append(l.numpy()) 
  
for s,l in test_data:
  testing_sentences.append(s.numpy().decode('utf8'))
  testing_labels.append(l.numpy())

# list to array
training_labels_final = np.array(training_labels) 
testing_labels_final = np.array(testing_labels)


In [ ]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(), # word, label로 인해 2D array를 1차원으로 바꿔야 함
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [ ]:

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
# encoding 한 것을 decoding 하기 위해
# { 'hello':1, 'world':2 } 단어-숫자 쌍을 뒤집어 주기
# { 1:'hello', 2:'world' }


def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3]))
print(training_sentences[3])

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received
This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.


In [ ]:
num_epochs = 10
model.fit(padded, 
          training_labels_final, 
          epochs=num_epochs, 
          validation_data=(testing_padded, testing_labels_final))

# accuracy= 1, valid=0.8로 overfitting

Epoch 1/10
782/782 [==============================] - 7s 8ms/step - loss: 0.5607 - accuracy: 0.7216 - val_loss: 0.4431 - val_accuracy: 0.8376
Epoch 2/10
782/782 [==============================] - 6s 8ms/step - loss: 0.3379 - accuracy: 0.8955 - val_loss: 0.4225 - val_accuracy: 0.8315
Epoch 3/10
782/782 [==============================] - 6s 8ms/step - loss: 0.1887 - accuracy: 0.9579 - val_loss: 0.4398 - val_accuracy: 0.8298
Epoch 4/10
782/782 [==============================] - 6s 8ms/step - loss: 0.1140 - accuracy: 0.9807 - val_loss: 0.5541 - val_accuracy: 0.8157
Epoch 5/10
782/782 [==============================] - 6s 8ms/step - loss: 0.0820 - accuracy: 0.9861 - val_loss: 0.5113 - val_accuracy: 0.8266
Epoch 6/10
782/782 [==============================] - 5s 7ms/step - loss: 0.0663 - accuracy: 0.9883 - val_loss: 0.5757 - val_accuracy: 0.8268
Epoch 7/10
782/782 [==============================] - 5s 7ms/step - loss: 0.0592 - accuracy: 0.9888 - val_loss: 0.5978 - val_accuracy: 0.8274
Epoch 

In [ ]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [ ]:
import io

# 벡터 파일로 만들면 3D 공간에 그 파일을 시각화할 수 있음
# https://projector.tensorflow.org/
# 여기에 해당 파일 올리기
# 왼쪽의 sphereize data 체크하면 data clustering도 볼 수 있음

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  # 메타데이터에는 그저 단어만 적어줌
  out_m.write(word + "\n")
  # 각 단어의 차원의 계수를 적음
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")

out_v.close()
out_m.close()

In [ ]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences([sentence])
print(sequence)

[[11, 64, 102, 12, 7, 478, 1200]]
